<a href="https://colab.research.google.com/github/taeheee-kim/deep_learning_practice/blob/master/%EC%A0%84%EC%B2%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0_epoch6_%EB%8F%8C%EB%A6%B0_%EB%AA%A8%EB%8D%B8%EC%9C%BC%EB%A1%9C_TTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Oct  6 17:32:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

albumentations==1.1.0 is successfully installed


## Run the example

### Import the required libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

In [5]:
import gc
import torch
import torch.utils.data as Data
import torch.nn as nn
from torchvision import transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


import torch # 파이토치
import random
import os


In [6]:

# 시드 값 고정
seed = 10
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


import cv2
from torch.utils.data import Dataset  # 데이터 생성을 위한 클래스
import numpy as np
import glob


class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()  # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    # 데이터 세트 크기 반환 메서드
    def __len__(self):
        return len(self.df)

    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        
        img_id = self.df.iloc[idx, 0] # 이미지 ID
        img_path = self.img_dir + str(img_id) + '.png' # 이미지 파일 경로
        print(img_path)
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image=image)['image']


        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃 값(label)도 함께 반환
        if self.is_test:
            return image
        else:
            # 타깃 값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:11])
            return image, label

In [7]:
# 이미지 변환을 위한 모듈
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(400, 600), # 이미지 크기 조절
    # 밝기 및 대비 조절 
    A.RandomBrightnessContrast(brightness_limit=0.1, 
                               contrast_limit=0.1, p=0.5),
    A.VerticalFlip(p=0.5), # 상하 대칭 변환
    A.HorizontalFlip(p=0.5), # 좌우 대칭 변환
    # 이동, 스케일링, 회전 변환 
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=25, p=0.7),
    # 양각화, 날카로움, 블러 효과 
    A.OneOf([A.Emboss(p=1),
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.5),
    A.PiecewiseAffine(p=0.5), # 어파인 변환 
    A.Normalize(), # 정규화 변환 
    ToTensorV2() # 텐서로 변환
])

# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(400, 600),
    A.Normalize(),
    ToTensorV2()
])

In [8]:
def seed_worker(worker_id): #시드를 고정해주는 역할
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [9]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스


In [10]:
!pip install efficientnet-pytorch==0.7.1

In [11]:
!pip install transformers

In [12]:
from efficientnet_pytorch import EfficientNet # EfficientNet 모델


In [13]:
import torch.nn as nn # 신경망 모듈

# 손실함수
criterion = nn.CrossEntropyLoss()
model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10)
# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00007, weight_decay=0.0001)


Loaded pretrained weights for efficientnet-b6


In [14]:
#모델 이어서 돌리는 코드
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대

import copy
PATH = '/content/drive/MyDrive/Tomato/태희/'

#model = EfficientNet.from_pretrained('efficientnet-b6', num_classes=10) #하시던거 똑같이 유지
device = 'cuda'
model.load_state_dict(copy.deepcopy(torch.load(PATH + "epoch10.pt",device))) # 9, 14, 19, 24, 29 

model = model.to(device) # 장비 할당

batch_size = 6 #하시던거 똑같이 유지

In [15]:
data_path = '/content/drive/MyDrive/Tomato/'

test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [16]:
img_dir = '/content/drive/MyDrive/Tomato/test_images/'

dataset_TTA = ImageDataset(test, img_dir=img_dir, 
                           transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, 
                        shuffle=False, worker_init_fn=seed_worker,
                        generator=g)

In [17]:
num_TTA = 5 # TTA 횟수

preds_tta = np.zeros((len(test), 10)) # 예측 값 저장용 배열 초기화 (TTA용)

# TTA를 적용해 예측 (5에폭)
for i in range(num_TTA):
    print(i)
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
            # 타깃 예측 확률
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/drive/MyDrive/Tomato/test_images/1927.png
/content/drive/MyDrive/Tomato/test_images/1928.png
/content/drive/MyDrive/Tomato/test_images/1929.png
/content/drive/MyDrive/Tomato/test_images/1930.png
/content/drive/MyDrive/Tomato/test_images/1931.png
/content/drive/MyDrive/Tomato/test_images/1932.png
/content/drive/MyDrive/Tomato/test_images/1933.png
/content/drive/MyDrive/Tomato/test_images/1934.png
/content/drive/MyDrive/Tomato/test_images/1935.png
/content/drive/MyDrive/Tomato/test_images/1936.png
/content/drive/MyDrive/Tomato/test_images/1937.png
/content/drive/MyDrive/Tomato/test_images/1938.png
/content/drive/MyDrive/Tomato/test_images/1939.png
/content/drive/MyDrive/Tomato/test_images/1940.png
/content/drive/MyDrive/Tomato/test_images/1941.png
/content/drive/MyDrive/Tomato/test_images/1942.png
/content/drive/MyDrive/Tomato/test_images/1943.png
/content/drive/MyDrive/Tomato/test_images/1944.png
/content/drive/MyDrive/Tomato/test_images/1945

In [18]:
preds_tta /= num_TTA 

In [19]:
submission_tta = submission.copy() 

submission_tta[['Tomato_D01','Tomato_D04','Tomato_D05','Tomato_D07','Tomato_D08','Tomato_D09','Tomato_H','Tomato_P03','Tomato_P05','Tomato_R01']] = preds_tta

In [20]:
submission_tta

,file_name,answer,Tomato_D01,Tomato_D04,Tomato_D05,Tomato_D07,Tomato_D08,Tomato_D09,Tomato_H,Tomato_P03,Tomato_P05,Tomato_R01
0,0.png,0,9.907558e-01,2.875468e-04,6.486971e-05,1.865416e-03,1.942927e-03,1.567148e-04,0.003137,4.438138e-04,1.023322e-03,3.228803e-04
1,1.png,0,1.278833e-05,4.620030e-05,9.964668e-01,6.085991e-05,3.091809e-05,1.767620e-05,0.001062,1.494909e-04,2.150998e-03,2.702324e-06
2,2.png,0,1.075352e-10,5.079409e-12,4.800647e-13,6.952983e-09,1.175584e-10,6.262704e-10,1.000000,1.623011e-07,2.734196e-10,2.538112e-10
3,3.png,0,7.936373e-08,9.973270e-01,2.566339e-08,1.834219e-07,1.076397e-06,1.190464e-06,0.002663,6.296620e-06,8.900468e-07,8.046526e-08
4,4.png,0,2.308548e-05,4.888411e-05,9.996947e-01,1.180900e-05,4.793181e-05,2.572001e-05,0.000027,7.617466e-07,1.109546e-04,9.256922e-06
...,...,...,...,...,...,...,...,...,...,...,...,...
3458,3458.png,0,9.997766e-01,1.833039e-05,3.662429e-05,5.351626e-06,4.463794e-05,2.927069e-06,0.000001,3.423802e-06,1.068794e-04,3.762355e-06
3459,3459.png,0,3.049610e-08,2.174291e-06,1.172846e-06,2.275294e-06,7.011966e-06,9.138603e-07,0.999986,8.512579e-08,2.813030e-08,1.610267e-07
3460,3460.png,0,3.626509e-05,2.158572e-03,5.886044e-04,3.037618e-04,1.305843e-03,3.085417e-04,0.120743,8.729056e-01,1.630889e-03,1.889408e-05
3461,3461.png,0,4.478475e-08,3.742142e-06,4.004953e-06,4.850154e-06,3.805032e-07,2.052179e-06,0.999961,2.242416e-05,9.530186e-07,5.013978e-07


In [21]:
id = submission_tta["file_name"]

In [22]:
pred_idx = np.argmax(preds_tta, axis=1)  #확률 중 가장 높은 값의 인덱스를 0 ~ 9까지 반환

In [23]:
pred_idx

array([0, 2, 6, ..., 7, 6, 6])

In [24]:
df = pd.DataFrame({'file_name':id, 'answer':pred_idx})

In [25]:
df 

,file_name,answer
0,0.png,0
1,1.png,2
2,2.png,6
3,3.png,1
4,4.png,2
...,...,...
3458,3458.png,0
3459,3459.png,6
3460,3460.png,7
3461,3461.png,6


In [26]:
df.to_csv('/content/drive/My Drive/Tomato/태희/submission_tta_epoch10.csv', index=False)